# Data Processing

Returns 

In [1]:
import pandas as pd
import numpy as np
import html
import ast
from pprint import pprint
from tqdm import tqdm
import re
import json
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_colwidth', 50)

In [2]:
data = pd.read_csv('zomato.csv')

## Determine How Deduplication Picks Records
- Highest priority for the same listed_in and location
- Second highest priority for more votes

In [3]:
data['votes'] = data['votes'].astype(int)

data['Consistent_Location'] = False
data.loc[data['location'] == data['listed_in(city)'], 'Consistent_Location'] = True
data = data.sort_values(by=['Consistent_Location','votes'], ascending = False)

## Column String Cleanup

In [5]:
x = data['rate'].str.replace(' ', '').str.replace('/5', '').str.replace('NEW', '')

print(len(x.groupby(x).size()))

data['rate'] = data['rate'].str.replace(' ', '').str.replace('/5', '').str.replace('NEW', '')

33


### Cleaning up reviews

In [6]:
def clean_review(review):
    review = html.unescape(review)
    review = review.encode().decode('unicode_escape')
    review = review.replace("\n", "")
    try:
        review = ast.literal_eval(review)
    except Exception as e:
        review = review.strip('][')
        review = review.strip('()')
        review = review.split('), (')
        try:
            review = [(item[:11].strip("'"), ' '.join(re.findall(r"[\w%\-.']+", item[14:-1].strip('"')))) for item in review]
        except Exception as e:
            print(review)
    return review

In [7]:
for column in list(data.columns.values):
    data[column] = data[column].apply(str)

In [8]:
data_url_review = data[['url', 'name','address','reviews_list']]

In [9]:
data_url_review.reviews_list[2]

'[(\'Rated 3.0\', "RATED\\n  Ambience is not that good enough and it\'s not a pocket friendly cafe and the quantity is not that good and desserts are too good enough ??.."), (\'Rated 3.0\', "RATED\\n \\nWent there for a quick bite with friends.\\nThe ambience had more of corporate feel. I would say it was unique.\\nTried nachos, pasta churros and lasagne.\\n\\nNachos were pathetic.( Seriously don\'t order)\\nPasta was okayish.\\nLasagne was good.\\nNutella churros were the best.\\nOverall an okayish experience!\\nPeace ??"), (\'Rated 4.0\', "RATED\\n  First of all, a big thanks to the staff of this Cafe. Very polite and courteous.\\n\\nI was there 15mins before their closing time. Without any discomfort or hesitation, the staff welcomed me with a warm smile and said they\'re still open, though they were preparing to close the cafe for the day.\\n\\nQuickly ordered the Thai green curry, which is served with rice. They got it for me within 10mins, hot and freshly made.\\n\\nIt was tasty 

### Removing Duplicates

In [10]:
data_cleaned = data.groupby(['name', 'address']).agg({'online_order' : ','.join,
         'book_table' : ','.join,
         'rate': ','.join,
         'votes': ','.join,
         'location': ','.join, 
         'rest_type': ','.join,
         'dish_liked': ','.join,
         'cuisines': ','.join,
         'approx_cost(for two people)': ','.join,
         'menu_item': ','.join,
         'listed_in(city)': ','.join,
         'listed_in(type)': ','.join}).reset_index()

In [11]:
data_cleaned.head()

,name,address,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,approx_cost(for two people),menu_item,listed_in(city),listed_in(type)
0,#FeelTheROLL,"Opposite Mantri Commercio, Outer Ring Road, De...","No,No","No,No","3.4,3.4","7,7","Bellandur,Bellandur","Quick Bites,Quick Bites","nan,nan","Fast Food,Fast Food","200,200","[],[]","Bellandur,Sarjapur Road","Delivery,Delivery"
1,#L-81 Cafe,"Sector 6, HSR Layout, HSR","Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes","No,No,No,No,No,No,No,No,No","3.9,3.9,3.9,3.9,3.9,3.9,3.9,3.9,3.9","48,48,48,48,48,48,48,48,48","HSR,HSR,HSR,HSR,HSR,HSR,HSR,HSR,HSR","Quick Bites,Quick Bites,Quick Bites,Quick Bite...","Burgers,Burgers,Burgers,Burgers,Burgers,Burger...","Fast Food, Beverages,Fast Food, Beverages,Fast...","400,400,400,400,400,400,400,400,400","[],[],[],[],[],[],[],[],[]","HSR,HSR,BTM,BTM,Koramangala 4th Block,Koramang...","Delivery,Dine-out,Delivery,Dine-out,Delivery,D..."
2,#Vibes Restro,"Marasur Gate, Chandapura - Anekal Road, Near A...","No,No,No","No,No,No","nan,nan,nan","0,0,0","Electronic City,Electronic City,Electronic City","Casual Dining,Casual Dining,Casual Dining","nan,nan,nan","Continental, Chinese, Italian,Continental, Chi...","700,700,700","[],[],[]","Electronic City,Electronic City,Electronic City","Buffet,Delivery,Dine-out"
3,#refuel,"7, Ground Floor, RR Commercial Complex, Akshay...","Yes,Yes,Yes","No,No,No","3.7,3.7,3.7","37,37,37","Bannerghatta Road,Bannerghatta Road,Bannerghat...","Cafe,Cafe,Cafe","Thick Shakes, Sandwiches, Pasta, Mocktails,Thi...","Cafe, Beverages,Cafe, Beverages,Cafe, Beverages","400,400,400","['Kit Kat Thick Shake', 'Ferrero Rocher Thick ...","Bannerghatta Road,Bannerghatta Road,Bannerghat...","Cafes,Delivery,Dine-out"
4,'Brahmins' Thatte Idli,"19, 1st main, 2nd cross, 3rd stage, 3rd block,...",Yes,No,nan,0,Basaveshwara Nagar,Quick Bites,nan,South Indian,100,"['Masala Dosa', 'Set Dosa', 'Shavige Bhath', '...",Rajajinagar,Dine-out


### Adding url and reviews

In [12]:
data_cleaned = data_cleaned.merge(data_url_review, how='left', on=['name', 'address']).drop_duplicates(['address','name'])

In [13]:
columns = list(data_cleaned.columns.values)
columns.pop()
columns.pop()

'url'

### Taking Highest Priority Row Value or Combining Values

In [14]:
data_cleaned.head(1)

,name,address,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,approx_cost(for two people),menu_item,listed_in(city),listed_in(type),url,reviews_list
0,#FeelTheROLL,"Opposite Mantri Commercio, Outer Ring Road, De...","No,No","No,No","3.4,3.4","7,7","Bellandur,Bellandur","Quick Bites,Quick Bites","nan,nan","Fast Food,Fast Food","200,200","[],[]","Bellandur,Sarjapur Road","Delivery,Delivery",https://www.zomato.com/bangalore/feeltheroll-b...,"[('Rated 5.0', ""RATED\n Had an egg chicken ro..."


In [15]:
def leading_char_removal(entry, char):
    try:
        if entry[0] == char:
            entry = entry[1:]
    except IndexError:
        pass
    
    try:
        if entry[-1] == char:
            entry = entry[:-1]
    except IndexError:
        pass
    
    return(entry)

In [17]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2)))

columns_first = ['online_order', 'book_table','rate','votes','location','approx_cost(for two people)']
columns_merge = Diff(columns, columns_first)

columns_merge.remove('address')

In [20]:
columns_merge

['listed_in(city)',
 'rest_type',
 'menu_item',
 'dish_liked',
 'listed_in(type)',
 'cuisines']

In [21]:
def clean_dup_entry_first(entry):
    entry = leading_char_removal(entry, ',')
    entry = entry.split(',')
    entry = list(set(entry))
    
    new_entry = list()
    for item in entry:
        item = item.strip()
        item = leading_char_removal(item, ',')
        item = leading_char_removal(item, '[')
        item = leading_char_removal(item, ']')
        
        new_entry.append(item)
    entry = new_entry
    
    entry = entry[0]
    return entry

In [22]:
def clean_dup_entry(entry):
    entry = leading_char_removal(entry, ',')
    entry = entry.split(',')
    entry = list(set(entry))
    
    new_entry = list()
    for item in entry:
        item = item.strip()
        item = leading_char_removal(item, ',')
        item = leading_char_removal(item, '[')
        item = leading_char_removal(item, ']')
        
        new_entry.append(item)
    entry = new_entry
    entry = list(set(entry))
    try:
        entry.remove('')
    except ValueError:
        pass
    entry = ','.join(entry)
    return entry


print('Cleaning Data in', columns)

for column in columns_merge:
    data_cleaned[column] = data_cleaned[column].apply(clean_dup_entry)
    
for column in columns_first:
    data_cleaned[column] = data_cleaned[column].apply(clean_dup_entry_first)

Cleaning Data in ['name', 'address', 'online_order', 'book_table', 'rate', 'votes', 'location', 'rest_type', 'dish_liked', 'cuisines', 'approx_cost(for two people)', 'menu_item', 'listed_in(city)', 'listed_in(type)']


In [23]:
def remove_comma(item):
    item = leading_char_removal(item, ',')
    return(item)

data_cleaned['menu_item'] = data_cleaned['menu_item'].apply(remove_comma)

In [24]:
data_cleaned = data_cleaned.replace('nan', np.NaN)

In [48]:
def data_cleaning_review(data_cleaned, row_view: int = 10):
    print('Columns:', data_cleaned.columns.values,'\n')
    print('With rows of', len(data_cleaned), '\n')
    for col in data_cleaned.columns.values:
        print('---------------Column of:', col, '---')
        print('Dtype of', data_cleaned[col].dtypes )

        x = data_cleaned[col]
        x = x.groupby(x).size().sort_values()
        print('Number of Unique Values:', len(x))

        print('\n First 5 Items', x.head(row_view),'\n')
        print('Last 5 Items', x.tail(row_view))

        print('\n')
        
data_cleaning_review(data_cleaned, row_view = 10)

## Convert Merged Strings

#### online_order

In [33]:
# data_cleaned['online_order'] = data_cleaned['online_order'].replace('Yes,No', 'Yes')
data_cleaned['online_order'] = data_cleaned['online_order'].replace('Yes', 1).replace('No', 0).astype('bool')

In [34]:
x = data_cleaned['online_order']
x.groupby(x).size()

online_order
False    6187
True     6312
Name: online_order, dtype: int64

#### rate

In [35]:
data_cleaned['rate'] = data_cleaned['rate'].replace('-', np.NaN).replace('', np.NaN).astype(float)

x = data_cleaned['rate']
x.groupby(x).size().index.values

array([1.8, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. , 3.1,
       3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4,
       4.5, 4.6, 4.7, 4.8, 4.9])

#### book_table

In [36]:
#data_cleaned['book_table'] = data_cleaned['book_table'].replace('Yes, No', 'No')
data_cleaned['book_table'] = data_cleaned['book_table'].replace('Yes', 1).replace('No', 0).astype('bool')

In [37]:
x = data_cleaned['book_table']
x.groupby(x).size().head()

book_table
False    11558
True       941
Name: book_table, dtype: int64

#### votes
Different from Ratings, so more work needs to be done on that field.

In [38]:
data_cleaned['votes'] = data_cleaned['votes'].astype('int')

In [39]:
x = data_cleaned['votes'].astype('int')
len(x.groupby(x).size())

1199

In [42]:
x.groupby(x).size().sort_values()

votes
16345       1
518         1
1227        1
1224        1
526         1
528         1
530         1
531         1
532         1
536         1
         ... 
12        159
5         162
11        164
9         167
8         173
10        189
7         270
6         293
4         378
0        3042
Name: votes, Length: 1199, dtype: int64

#### location
Field has already been cleaned

#### listed_in(city)
Field has already been cleaned

#### rest_type
Field has already been cleaned

#### dish_liked
Field has been cleaned

#### cuisines
Field has been cleaned

#### approx_cost(for two people)

In [43]:
data_cleaned['approx_cost(for two people)'] = data_cleaned['approx_cost(for two people)'].astype('float')

In [44]:
x = data_cleaned['approx_cost(for two people)'].astype('float')
x.groupby(x).size().sort_values()

approx_cost(for two people)
330.0       1
140.0       1
130.0       1
190.0       1
560.0       1
240.0       1
469.0       1
60.0        1
40.0        1
6.0         1
         ... 
350.0     439
800.0     456
150.0     623
1.0       802
250.0     842
600.0     852
500.0    1273
200.0    1406
300.0    1905
400.0    1909
Name: approx_cost(for two people), Length: 40, dtype: int64

In [47]:
# Review the Data
data_cleaning_review(data_cleaned)

In [46]:
data_cleaned.to_csv("zomato_cleaned.csv")
data_cleaned.drop('reviews_list', axis = 1).to_csv("zomato_cleaned_no_reviews.csv")

## Computer Readable Format Conversion (Not Working)

In [49]:
all_columns = data_cleaned.columns.tolist()
all_columns

['name',
 'address',
 'online_order',
 'book_table',
 'rate',
 'votes',
 'location',
 'rest_type',
 'dish_liked',
 'cuisines',
 'approx_cost(for two people)',
 'menu_item',
 'listed_in(city)',
 'listed_in(type)',
 'url',
 'reviews_list']

In [50]:
non_categorical_fields = ['name','address','online_order','book_table','rate','votes','location','approx_cost(for two people)','listed_in(city)','url','reviews_list']
non_categorical_fields

['name',
 'address',
 'online_order',
 'book_table',
 'rate',
 'votes',
 'location',
 'approx_cost(for two people)',
 'listed_in(city)',
 'url',
 'reviews_list']

In [51]:
categorical_fields = Diff(all_columns, non_categorical_fields)
categorical_fields

index_listed_in = categorical_fields.index("listed_in(type)")

categorical_fields[0], categorical_fields[index_listed_in] = categorical_fields[index_listed_in], categorical_fields[0]

categorical_fields

['listed_in(type)', 'menu_item', 'dish_liked', 'rest_type', 'cuisines']

In [52]:
computer_readable_data_cleaned = data_cleaned[non_categorical_fields]
print(len(computer_readable_data_cleaned))
computer_readable_data_cleaned.head(1)

12499


,name,address,online_order,book_table,rate,votes,location,approx_cost(for two people),listed_in(city),url,reviews_list
0,#FeelTheROLL,"Opposite Mantri Commercio, Outer Ring Road, De...",False,False,3.4,7,Bellandur,200.0,"Sarjapur Road,Bellandur",https://www.zomato.com/bangalore/feeltheroll-b...,"[('Rated 5.0', ""RATED\n Had an egg chicken ro..."


In [61]:
categorical_fields

['listed_in(type)', 'menu_item', 'dish_liked', 'rest_type', 'cuisines']

In [63]:
# Script Below Did Not Resolve, very high computational cost

computer_readable_files = list()
#computer_readable_data_cleaned = computer_readable_data_cleaned[['name','address']]

for column in categorical_fields:
    print('Processing', column)
    column_one_hot_data = (data_cleaned[column].str.split('\s*,\s*', expand=True)
       .stack()
       .str.get_dummies()
       .sum(level=0).add_prefix(column + '_'))
    column_one_hot_data = column_one_hot_data.fillna(0)
    
    # Trying to convert float 1.0 to 1, but didn't work
    # column_one_hot_data.columns = column_one_hot_data.columns.str.strip()
    # column_one_hot_data = column_one_hot_data.convert_dtypes()
    
    # computer_readable_data_cleaned = computer_readable_data_cleaned.join(column_one_hot_data)
    # computer_readable_data_cleaned = computer_readable_data_cleaned.fillna(0)
    
    column_one_hot_data.to_csv(column + '.csv')

Processing listed_in(type)
Processing menu_item


KeyboardInterrupt: 

In [ ]:
listed_in_data = computer_readable_data_cleaned.copy(deep = True)

In [56]:
listed_in_data

,name,address,listed_in(type)_Buffet,listed_in(type)_Cafes,listed_in(type)_Delivery,listed_in(type)_Desserts,listed_in(type)_Dine-out,listed_in(type)_Drinks & nightlife,listed_in(type)_Pubs and bars
0,#FeelTheROLL,"Opposite Mantri Commercio, Outer Ring Road, De...",0,0,1,0,0,0,0
2,#L-81 Cafe,"Sector 6, HSR Layout, HSR",0,0,1,0,1,0,0
11,#Vibes Restro,"Marasur Gate, Chandapura - Anekal Road, Near A...",1,0,1,0,1,0,0
14,#refuel,"7, Ground Floor, RR Commercial Complex, Akshay...",0,1,1,0,1,0,0
17,'Brahmins' Thatte Idli,"19, 1st main, 2nd cross, 3rd stage, 3rd block,...",0,0,0,0,1,0,0
18,1 Fahreheit,"Nishika Arcade, Nanjappa Main Road, Next to Jo...",0,0,0,1,0,0,0
19,1000 B.C,"16, 17th A Main, Koramangala 5th Block, Bangalore",0,0,1,0,0,0,0
25,100ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ°C,"688, Thanish Corner, 7th Main, 10th Cross, 2nd...",0,0,0,0,1,0,0
28,11 to 11 Express Biriyanis,"Near Velankani Bus Stop, Electroniccity Phase ...",0,0,1,0,1,0,0
30,1131 Bar + Kitchen,"100 Feet Road, HAL 2nd Stage, Indiranagar, Ban...",0,0,0,0,1,1,1
